# Part I. Preparation

First, we load the CIFAR-100 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

In [1]:
# Add official website of pytorch

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F

import numpy as np

In [2]:
NUM_TRAIN = 49000
batch_size= 64

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.

#===========================================================================#
# You should try changing the transform for the training data to include    #
# data augmentation such as RandomCrop and HorizontalFlip                    #
# when running the final part of the notebook where you have to achieve     #
# as high accuracy as possible on CIFAR-100.                                #
# Of course you will have to re-run this block for the effect to take place #
#===========================================================================#
train_transform = transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-100
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar100_train = dset.CIFAR100('./datasets/cifar100', train=True, download=True,
                             transform=train_transform)
loader_train = DataLoader(cifar100_train, batch_size=batch_size, num_workers=2,
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar100_val = dset.CIFAR100('./datasets/cifar100', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar100_val, batch_size=batch_size, num_workers=2,
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar100_test = dset.CIFAR100('./datasets/cifar100', train=False, download=True,
                            transform=transform)
loader_test = DataLoader(cifar100_test, batch_size=batch_size, num_workers=2)

100%|██████████| 169M/169M [00:38<00:00, 4.43MB/s]


You have an option to **use GPU by setting the flag to True below** (recommended). It is not necessary to use GPU for this assignment. Note that if your computer does not have CUDA enabled, `torch.cuda.is_available()` will return False and this notebook will fallback to CPU mode.
**You can run on GPU on datahub.**

The global variables `dtype` and `device` will control the data types throughout this assignment.

In [3]:
USE_GPU = True
num_class = 100
dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss


print('using device:', device)

using device: cuda


In [4]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print('Before flattening: ', x)
    print('After flattening: ', flatten(x))

test_flatten()

Before flattening:  tensor([[[[ 0,  1],
          [ 2,  3],
          [ 4,  5]]],


        [[[ 6,  7],
          [ 8,  9],
          [10, 11]]]])
After flattening:  tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])


In [5]:
def random_weight(shape):
    """
    Create random Tensors for weights; setting requires_grad=True means that we
    want to compute gradients for these Tensors during the backward pass.
    We use Kaiming normalization: sqrt(2 / fan_in)
    """
    if len(shape) == 2:  # FC weight
        fan_in = shape[0]
    else:
        fan_in = np.prod(shape[1:]) # conv weight [out_channel, in_channel, kH, kW]
    # randn is standard normal distribution generator.
    w = torch.randn(shape, device=device, dtype=dtype) * np.sqrt(2. / fan_in)
    w.requires_grad = True
    return w

def zero_weight(shape):
    return torch.zeros(shape, device=device, dtype=dtype, requires_grad=True)

# create a weight of shape [3 x 5]
# you should see the type `torch.cuda.FloatTensor` if you use GPU.
# Otherwise it should be `torch.FloatTensor`
random_weight((3, 5))

tensor([[ 0.5881, -0.9331, -0.5931, -1.0455, -0.0518],
        [ 1.1819,  0.6901, -0.2150, -0.5044, -0.1865],
        [-0.4173,  0.2111,  0.0749, -0.3254,  1.4280]], device='cuda:0',
       requires_grad=True)

### Check Accuracy


### Module API: Check Accuracy
Given the validation or test set, we can check the classification accuracy of a neural network.

This version is slightly different from the one in part II. You don't manually pass in the parameters anymore.

In [6]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    return acc

### Module API: Training Loop
We also use a slightly different training loop. Rather than updating the values of the weights ourselves, we use an Optimizer object from the `torch.optim` package, which abstract the notion of an optimization algorithm and provides implementations of most of the algorithms commonly used to optimize neural networks.

In [7]:
def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.

    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for

    Returns: The accuracy of the model
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

        print('Epoch %d, Iteration %d, loss = %.4f' % (e, t + 1, loss.item()))
        check_accuracy_part34(loader_val, model)
        print()

    return check_accuracy_part34(loader_val, model)

# Resnet10 Implementation


In [8]:

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, batch_use = False):
        super(BasicBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, (3,3), stride=1, padding=1, bias = True)
        # only use Batchnorm in part 2
        self.BatchNorm1 = nn.BatchNorm2d(out_channels) if batch_use else nn.Identity()
        self.relu1 = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(out_channels, out_channels, (3,3), stride=1, padding=1, bias = True)
        self.BatchNorm2 = nn.BatchNorm2d(out_channels) if batch_use else nn.Identity()
        self.relu2 = nn.ReLU(inplace=True)

        self.shortcut = nn.Conv2d(in_channels, out_channels, (1,1), stride=1, padding=0, bias = True)
        if in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, (1,1), stride=1, padding=0, bias = True),
                # nn.ReLU(inplace=True)

            )
            self.shortcut[0].weight.data = random_weight((out_channels, in_channels, 1, 1))
            self.shortcut[0].bias.data = zero_weight((out_channels,))

        self.conv1.weight.data = random_weight((out_channels, in_channels, 3, 3))
        self.conv1.bias.data = zero_weight((out_channels,))
        self.conv2.weight.data = random_weight((out_channels, out_channels, 3, 3))
        self.conv2.bias.data = zero_weight((out_channels,))

    def forward(self, x):
        identity = x if self.shortcut is None else self.shortcut(x)
        out = self.conv1(x)
        # only use Batchnorm in part 2
        out = self.BatchNorm1(out)

        out = self.relu1(out)
        out = self.conv2(out)
        # only use Batchnorm in part 2
        out = self.BatchNorm2(out)

        out += identity
        out = self.relu2(out)
        return out
class ResNet(nn.Module):

    def __init__(self, in_channels, num_classes = 100, batch_use = False):
        super(ResNet, self).__init__()

        # conv1
        self.conv1 = nn.Conv2d(in_channels, 64, (7,7), stride=2, padding=1, bias = True)
        # only use Batchnorm in part 2
        self.BatchNorm = nn.BatchNorm2d(64) if batch_use else nn.Identity()

        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d((3,3), stride=2, padding=1)

        # conv2_x
        self.conv2_x = BasicBlock(64, 128, batch_use)

        # conv3_3
        self.conv3_x = BasicBlock(128, 256, batch_use)

        # conv4_x
        self.conv4_x = BasicBlock(256, 512, batch_use)

        # conv5_x
        self.conv5_x = BasicBlock(512, 512,batch_use)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        # fc
        self.fc = nn.Linear(512, num_classes)

        self.conv1.weight.data = random_weight((64, in_channels, 7, 7))
        self.conv1.bias.data = zero_weight((64,))
        self.fc.weight.data = random_weight((num_classes, 512))
        self.fc.bias.data = zero_weight((num_classes,))
    def forward(self,x):
        x = self.conv1(x)
        # only use Batchnorm in part 2
        x = self.BatchNorm(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avg_pool(x)
        x = flatten(x)
        x = self.fc(x)
        return x



In [9]:
learning_rate = 1e-3

model = None
optimizer = None

model = ResNet(3, num_class)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


train_part34(model, optimizer, epochs=100)


Epoch 0, Iteration 766, loss = 3.9901
Checking accuracy on validation set
Got 123 / 1000 correct (12.30)

Epoch 1, Iteration 766, loss = 2.8448
Checking accuracy on validation set
Got 229 / 1000 correct (22.90)

Epoch 2, Iteration 766, loss = 2.1132
Checking accuracy on validation set
Got 294 / 1000 correct (29.40)

Epoch 3, Iteration 766, loss = 2.8710
Checking accuracy on validation set
Got 337 / 1000 correct (33.70)

Epoch 4, Iteration 766, loss = 2.1469
Checking accuracy on validation set
Got 360 / 1000 correct (36.00)

Epoch 5, Iteration 766, loss = 1.8363
Checking accuracy on validation set
Got 373 / 1000 correct (37.30)

Epoch 6, Iteration 766, loss = 1.7559
Checking accuracy on validation set
Got 409 / 1000 correct (40.90)

Epoch 7, Iteration 766, loss = 1.4281
Checking accuracy on validation set
Got 408 / 1000 correct (40.80)

Epoch 8, Iteration 766, loss = 1.2865
Checking accuracy on validation set
Got 405 / 1000 correct (40.50)

Epoch 9, Iteration 766, loss = 0.8897
Checking

0.353

## With BatchNorm

In [10]:
learning_rate = 1e-3

model = None
optimizer = None

################################################################################
# TODO: InstantiateResnet with BatchNorm                                       #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
model = ResNet(3, num_class, batch_use = True)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE
################################################################################


train_part34(model, optimizer, epochs=100)


Epoch 0, Iteration 766, loss = 3.0100
Checking accuracy on validation set
Got 302 / 1000 correct (30.20)

Epoch 1, Iteration 766, loss = 2.6068
Checking accuracy on validation set
Got 398 / 1000 correct (39.80)

Epoch 2, Iteration 766, loss = 2.1051
Checking accuracy on validation set
Got 431 / 1000 correct (43.10)

Epoch 3, Iteration 766, loss = 1.5909
Checking accuracy on validation set
Got 483 / 1000 correct (48.30)

Epoch 4, Iteration 766, loss = 1.5256
Checking accuracy on validation set
Got 512 / 1000 correct (51.20)

Epoch 5, Iteration 766, loss = 1.1340
Checking accuracy on validation set
Got 509 / 1000 correct (50.90)

Epoch 6, Iteration 766, loss = 0.6098
Checking accuracy on validation set
Got 502 / 1000 correct (50.20)

Epoch 7, Iteration 766, loss = 0.8621
Checking accuracy on validation set
Got 508 / 1000 correct (50.80)

Epoch 8, Iteration 766, loss = 0.4279
Checking accuracy on validation set
Got 537 / 1000 correct (53.70)

Epoch 9, Iteration 766, loss = 0.3440
Checking

0.498

## Discussion on BatchNorm
### TODO: Write your answer here
### 1.Validation Accuracy:
#### Without BatchNorm :
Starts at 8.00% (Epoch 0) and improves steadily to 40.00% (Epoch 9), with a final validation accuracy of 39.40%.
The accuracy increases consistently but plateaus around 36-40% after Epoch 6, indicating limited further improvement.
#### With BatchNorm :
Starts at 25.90% (Epoch 0) and reaches 51.20% (Epoch 9), with a final validation accuracy of 51.00%.
The accuracy improves more rapidly and achieves significantly higher values, surpassing 50% by Epoch 6 and stabilizing around 50-51% in later epochs.

### 2.Convergence Speed :
#### Without BatchNorm :
The model improves slowly, taking until Epoch 9 to reach 40.00% accuracy. The loss reduction is less stable, and accuracy plateaus earlier.
#### With BatchNorm :
The model converges faster, reaching 41.60% accuracy by Epoch 2 (vs. 22.90% without BatchNorm) and continuing to improve until Epoch 9. The loss decreases more smoothly and rapidly.
### Summary of Performance :
With BatchNorm: Achieves higher validation accuracy (51.00% vs. 39.40%), lower training loss (0.3992 vs. 1.0206), faster convergence, and greater training stability.

Without BatchNorm: Performs worse, with slower convergence, higher loss, and lower final accuracy, plateauing at a lower performance level.

## Discuss effect of Batch Size
### TODO: Write your answer here
#### the effect of different batch sizes
1. As batch size increases from 32 to 64 and 128, validation accuracy decreases (53.40% > 49.30% ≈ 49.90%). Batch size 32 clearly outperforms larger batch sizes in both final accuracy and stability.
2. Batch size 32 converges steadily and maintains improvement. Batch size 64 converges quickly early but stalls with fluctuations. Batch size 128’s converges the fastest.

#### possible reasons
1. As batch size increases from 32 to 64 and 128, validation accuracy decreases (53.40% > 49.30% ≈ 49.90%) because larger batch sizes lead to more stable but less noisy gradients, potentially causing the model to converge to sharper minima with poorer generalization. Batch size 32 introduces more stochasticity, helping the model escape local minima and achieve better generalization, resulting in higher accuracy and stability.
2. Batch size 32 converges steadily due to balanced gradient noise, while 64 converges quickly but fluctuates as noise decreases, causing instability. Batch size 128 converges fastest with the most stable gradients but may overfit, limiting further improvement.


# CIFAR-100 open-ended challenge
## What I did

In the cell below you should write an explanation of what you did, any additional features that you implemented, and/or any graphs that you made in the process of training and evaluating your network.

In [11]:

model = None
optimizer = None

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, batch_use = False):
        super(BasicBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, (3,3), stride=1, padding=1, bias = True)
        # only use Batchnorm in part 2
        self.BatchNorm1 = nn.BatchNorm2d(out_channels) if batch_use else nn.Identity()
        self.relu1 = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(out_channels, out_channels, (3,3), stride=1, padding=1, bias = True)
        self.BatchNorm2 = nn.BatchNorm2d(out_channels) if batch_use else nn.Identity()
        self.relu2 = nn.ReLU(inplace=True)

        self.conv1.weight.data = random_weight((out_channels, in_channels, 3, 3))
        self.conv1.bias.data = zero_weight((out_channels,))
        self.conv2.weight.data = random_weight((out_channels, out_channels, 3, 3))
        self.conv2.bias.data = zero_weight((out_channels,))

    def forward(self, x):
        out = self.conv1(x)
        out = self.BatchNorm1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.BatchNorm2(out)
        out = self.relu2(out)
        return out

class NewDefinedNet(nn.Module):

    def __init__(self, in_channels, num_classes = 100, batch_use = False):
        super(NewDefinedNet, self).__init__()

        # conv1
        self.conv1 = nn.Conv2d(in_channels, 64, (7,7), stride=2, padding=1, bias = True)
        self.BatchNorm = nn.BatchNorm2d(64) if batch_use else nn.Identity()

        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d((3,3), stride=2, padding=1)

        # conv2_x

        self.conv2_x = nn.Sequential(
            BasicBlock(64, 256, batch_use),
            nn.Dropout(p=0.5),
            BasicBlock(256, 512, batch_use),
        )
        # conv3_3
        self.conv3_x = nn.Sequential(
            BasicBlock(512, 1024, batch_use),
            nn.Dropout(p=0.5),
            BasicBlock(1024, 2048, batch_use),
        )

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        # fc
        self.fc = nn.Linear(2048, num_classes)

        self.conv1.weight.data = random_weight((64, in_channels, 7, 7))
        self.conv1.bias.data = zero_weight((64,))
        self.fc.weight.data = random_weight((num_classes, 2048))
        self.fc.bias.data = zero_weight((num_classes,))
    def forward(self,x):
        x = self.conv1(x)

        x = self.BatchNorm(x)
        x = self.relu(x)
        # x = self.maxpool(x)
        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.avg_pool(x)
        x = flatten(x)
        x = self.fc(x)
        return x

batch_size = 32
learning_rate = 1e-3



loader_train = DataLoader(cifar100_train, batch_size=batch_size, num_workers=2,
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

loader_val = DataLoader(cifar100_val, batch_size=batch_size, num_workers=2,
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

loader_test = DataLoader(cifar100_test, batch_size=batch_size, num_workers=2)


model = NewDefinedNet(3, num_class, batch_use = True)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)




train_part34(model, optimizer, epochs=10)


Epoch 0, Iteration 1532, loss = 3.6260
Checking accuracy on validation set
Got 93 / 1000 correct (9.30)

Epoch 1, Iteration 1532, loss = 4.1110
Checking accuracy on validation set
Got 186 / 1000 correct (18.60)

Epoch 2, Iteration 1532, loss = 2.9544
Checking accuracy on validation set
Got 278 / 1000 correct (27.80)

Epoch 3, Iteration 1532, loss = 3.3307
Checking accuracy on validation set
Got 330 / 1000 correct (33.00)

Epoch 4, Iteration 1532, loss = 3.8524
Checking accuracy on validation set
Got 392 / 1000 correct (39.20)

Epoch 5, Iteration 1532, loss = 3.1999
Checking accuracy on validation set
Got 424 / 1000 correct (42.40)

Epoch 6, Iteration 1532, loss = 2.4103
Checking accuracy on validation set
Got 484 / 1000 correct (48.40)

Epoch 7, Iteration 1532, loss = 1.8405
Checking accuracy on validation set
Got 499 / 1000 correct (49.90)

Epoch 8, Iteration 1532, loss = 0.6801
Checking accuracy on validation set
Got 531 / 1000 correct (53.10)

Epoch 9, Iteration 1532, loss = 2.2312


0.528

TODO: Describe what you did

I implemented a custom neural network (NewDefinedNet) for CIFAR-10 classification, using a batch size of 32, a learning rate of 10^-3 , and the Adam optimizer. The network structure includes a 7x7 convolutional layer (conv1) followed by batch normalization (optional), ReLU activation, and a max-pooling layer (commented out for experimentation). It then uses two sequential blocks (conv2_x and conv3_x), each containing two BasicBlock units with increasing channels (64→256→512 and 512→1024→2048) and dropout (p=0.5) for regularization. Additional features include flexible batch normalization via the batch_use flag and random_weight, zero_weight to initialize training process.

## Test set -- run this only once

Now that we've gotten a result we're happy with, we test our final model on the test set (which you should store in best_model). Think about how this compares to your validation set accuracy.

In [12]:
best_model = model
check_accuracy_part34(loader_test, best_model)

Checking accuracy on test set
Got 5301 / 10000 correct (53.01)


0.5301